In [1]:
import duckdb
import leafmap
import pandas as pd

In [2]:
con = duckdb.connect()
con.install_extension("httpfs")
con.load_extension("httpfs")
con.install_extension("spatial")
con.load_extension("spatial")

In [3]:
url = 'C:/Users/vance/Downloads/ukspills.xlsx'

In [4]:
con.sql("""
        CREATE TABLE IF NOT EXISTS spills as
        SELECT * FROM ST_Read('C:/Users/vance/Downloads/ukspills_clean.xlsx')

""")
con.table("spills")

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬────────────────────┐
│ OBJECTID │       COMPANY        │         SITE         │ … │     CONSTITUENCY     │ riverBasinDistrict │
│  int32   │       varchar        │       varchar        │   │       varchar        │      varchar       │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼────────────────────┤
│        1 │ Dwr Cymru Welsh Wa…  │ Trevethin CSO        │ … │ Torfaen              │ Western Wales      │
│        2 │ Dwr Cymru Welsh Wa…  │ Cwmbran Somerset R…  │ … │ Torfaen              │ Western Wales      │
│        3 │ Dwr Cymru Welsh Wa…  │ PONTNEWYNYDD CSO     │ … │ Torfaen              │ Western Wales      │
│        4 │ Dwr Cymru Welsh Wa…  │ Bridge St CSO        │ … │ Brecon and Radnors…  │ Western Wales      │
│        5 │ Dwr Cymru Welsh Wa…  │ Woodlands (Horrock…  │ … │ Newport West         │ Western Wales      │
│        6 │ Dwr Cymru Welsh Wa…  │ C

In [5]:
con.sql('''
    DESCRIBE spills


''')

┌────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│    column_name     │ column_type │  null   │   key   │ default │ extra │
│      varchar       │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ OBJECTID           │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ COMPANY            │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ SITE               │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ WATERBODY          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ DURATION           │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ SPILL_NUMBER       │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ Eastings           │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ Northings          │ INTEGER     │ YES     │ NULL    │ NULL    │  NULL │
│ Latitude           │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ Longitude          │ DO

In [11]:
con.sql('''
    SELECT COMPANY, SUM(SPILL_NUMBER) AS NUMBER, ROUND(SUM(DURATION),1) AS SPILL_HOURS

    FROM spills
    GROUP BY COMPANY
    ORDER BY NUMBER DESC
''')

┌───────────────────────┬────────┬─────────────┐
│        COMPANY        │ NUMBER │ SPILL_HOURS │
│        varchar        │ int128 │   double    │
├───────────────────────┼────────┼─────────────┤
│ Dwr Cymru Welsh Water │  86277 │    602987.5 │
│ United Utilities      │  69245 │    425490.6 │
│ Yorkshire Water       │  54273 │    232053.6 │
│ Severn Trent Water    │  44765 │    249115.6 │
│ South West Water      │  37649 │    290271.3 │
│ Northumbrian Water    │  29697 │    107536.1 │
│ Wessex Water          │  21878 │    129957.4 │
│ Southern Water        │  16688 │    146819.6 │
│ Anglian Water         │  16082 │     89513.8 │
│ Thames Water          │   8014 │     74693.3 │
├───────────────────────┴────────┴─────────────┤
│ 10 rows                            3 columns │
└──────────────────────────────────────────────┘

In [13]:
import folium

# Get the spill locations
locations = con.sql('''
    SELECT latitude, longitude
    FROM spills
''')


In [16]:
m = leafmap.Map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [20]:
df = leafmap.csv_to_df('C:/Users/vance/Downloads/ukspills_csv.csv')
df

,OBJECTID,COMPANY,SITE,WATERBODY,DURATION,SPILL_NUMBER,Eastings,Northings,Latitude,Longitude,COUNTRY,AUTHORITY,CONSTITUENCY,riverBasinDistrict
0,1,Dwr Cymru Welsh Water,Trevethin CSO,Afon Lwyd - source to Mon and Brecon Canal,143.75,40.0,327675.0,201786.0,51.710127,-3.048159,Wales,Torfaen,Torfaen,Western Wales
1,2,Dwr Cymru Welsh Water,Cwmbran Somerset Rd CSO,Afon Lwyd - below Mon and Brecon Canal,157.00,50.0,330035.0,195910.0,51.657602,-3.012829,Wales,Torfaen,Torfaen,Western Wales
2,3,Dwr Cymru Welsh Water,PONTNEWYNYDD CSO,Afon Lwyd - source to Mon and Brecon Canal,2.75,11.0,327800.0,201616.0,51.708614,-3.046315,Wales,Torfaen,Torfaen,Western Wales
3,4,Dwr Cymru Welsh Water,Bridge St CSO,Usk conf Afon Crawnon to conf Gavenny R,2.50,8.0,321548.0,218084.0,51.855809,-3.140506,Wales,Powys,Brecon and Radnorshire,Western Wales
4,5,Dwr Cymru Welsh Water,Woodlands (Horrocks Close) SPS,NaN,69.00,25.0,330504.0,191745.0,51.620218,-3.005221,Wales,Newport,Newport West,Western Wales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16786,16787,Yorkshire Water,Not in Consents Database,Dearne Darfield STW to River Don,3.25,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16787,16788,Yorkshire Water,Not in Consents Database,Dearne Darfield STW to River Don,1.25,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16788,16789,Yorkshire Water,Not in Consents Database,Batley Beck from Source to River Calder,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16789,16790,Yorkshire Water,Chapel Land Kilham CSO*,Lowthorpe/Kelk/Foston Bks from Source to Frodi...,6.75,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
m.add_xy_data(df, x="Longitude", y="Latitude", layer_name="Shit Spills")
m

Map(bottom=982.0, center=[51.41015510202106, -27.08789069205523], controls=(ZoomControl(options=['position', '…